In [1]:
pwd()

"/home/zyu20/git/BulkLMM.jl/test/notebooks"

In [2]:
cd("..")

In [3]:
Threads.nthreads()

24

In [4]:
include("BXDdata_for_test.jl"); ## Load BXD data used for testing:

In [5]:
using LoopVectorization

In [6]:
include("../src/scan_LiteQTL.jl"); ## Load multithreaded functions to be tested:

In [7]:
include("../src/parallel_helpers.jl");

In [8]:
BLAS.set_num_threads(24)

In [9]:
BLAS.get_num_threads()

24

In [10]:
hsq_list = collect(0:10) * 0.1;

In [11]:
kinship_Gc = CSV.read("run-gemma/output/kinship.cXX.txt", DataFrame, delim = '\t', header = false) |> Matrix;

In [42]:
@time bulkscan_LODs = bulkscan(pheno, geno, kinship_Gc, hsq_list);

 10.172633 seconds (2.15 k allocations: 22.258 GiB, 9.42% gc time)


In [43]:
bulkscan_LODs[1:6, :]

6×35556 Matrix{Float64}:
 0.117494  0.121561  0.110276  0.0494144  …  0.0472748  0.355458  0.548261
 0.117494  0.121561  0.110276  0.0494144     0.0472748  0.355458  0.548261
 0.117494  0.121561  0.110276  0.0494144     0.0472748  0.355458  0.548261
 0.117494  0.121561  0.110276  0.0494144     0.0472748  0.355458  0.548261
 0.117494  0.121561  0.110276  0.0494144     0.0472748  0.355458  0.548261
 0.117494  0.121561  0.110276  0.0494144  …  0.0472748  0.355458  0.548261

In [44]:
gemma = "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64";

In [45]:
gemma_results = readdlm("run-gemma/output/gemma_results.txt.assoc.txt", '\t');

In [46]:
function p2lod(pval::Float64, df::Int64)
    
    lrs = invlogcdf(Chisq(df), log(1-pval))
    lod = lrs/(2*log(10))
    
    # return lrs
    return lod

end

p2lod (generic function with 1 method)

In [47]:
gemma_LODs = map(x -> p2lod(x, 1), gemma_results[2:end, 10]);

In [48]:
hcat(gemma_LODs, bulkscan_LODs[:, 7919])

7321×2 Matrix{Float64}:
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347286  0.0230375
 0.000347136  0.0230399
 0.0136122    0.149115
 ⋮            
 0.349491     0.346613
 0.349491     0.346613
 0.349491     0.346613
 0.150758     0.149648
 5.69796e-5   0.177024
 5.69796e-5   0.177024
 0.000387442  0.20417
 0.00241246   0.242513
 0.00212172   0.238575
 0.00567796   0.275557
 0.00288966   0.235776
 0.00288966   0.235776

In [68]:
scan_null_lod = scan_null(reshape(pheno[:, 7919], :, 1), geno, kinship; reml = true);

In [69]:
scan_alt_lod = scan_alt(reshape(pheno[:, 7919], :, 1), geno, kinship; reml = false);

In [70]:
hcat(gemma_LODs, scan_null_lod[3], scan_alt_lod[3], bulkscan_LODs[:, 7919])

7321×4 Matrix{Float64}:
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347286  0.00798216  0.00819776  0.0230375
 0.000347136  0.00798112  0.00819668  0.0230399
 0.0136122    0.00957719  0.00987864  0.149115
 ⋮                                    
 0.349491     0.0954055   0.0953535   0.346613
 0.349491     0.0954055   0.0953535   0.346613
 0.349491     0.0954055   0.0953535   0.346613
 0.150758     0.0125858   0.0115362   0.149648
 5.69796e-5   0.0075433   0.00826847  0.177024
 5.69796e-5   0.0075433   0.00826847  0.177024
 0.000387442  0.0140671   0.0151

In [65]:
sumSqDiff(reshape(gemma_LODs, :, 1), reshape(bulkscan_LODs[:, 7919], :, 1))

Inf